<a href="https://colab.research.google.com/github/pakyeon/deep-rl-class-ko/blob/main/notebooks/ko/unit4/unit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 4: PyTorch로 처음으로 딥 강화학습 알고리즘인 Reinforce 구현하기. 그리고 그 강인함을 테스트하기 💪

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail"/>

이 노트북에서는 처음으로 딥 강화학습 알고리즘인 Reinforce(몬테카를로 정책 경사라고도 불림)를 처음부터 직접 구현하게 됩니다.

Reinforce는 *정책 기반 방법(Policy-based method)*입니다: 행동 가치 함수 없이 **정책을 직접 최적화**하려는 딥 강화학습 알고리즘입니다.

정확히 말하자면, Reinforce는 *정책 경사 방법(Policy-gradient method)*이며, 이는 *정책 기반 방법*의 하위 클래스입니다. 이는 **경사 상승법을 사용해 최적의 정책 가중치를 추정함으로써 정책을 직접 최적화**하려는 방법입니다.

강인함을 테스트하기 위해, 우리는 이 알고리즘을 두 가지 간단한 환경에서 학습시킬 것입니다:
- Cartpole-v1  
- PixelcopterEnv

⬇️ **이 노트북의 마지막에 여러분이 달성하게 될 예시입니다.** ⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Environments"/>


### 🎮 환경:

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 강화학습 라이브러리:

- Python  
- PyTorch

튜토리얼을 지속적으로 개선하고 있으니, **이 노트북에서 문제가 발견되면** [GitHub 저장소에 이슈를 등록해 주세요](https://github.com/huggingface/deep-rl-class/issues).

## 이 노트북의 목표 🏆

이 노트북을 마치면 여러분은:

- **PyTorch를 사용해 Reinforce 알고리즘을 처음부터 직접 구현**할 수 있습니다.  
- **간단한 환경을 통해 에이전트의 강인함을 테스트**할 수 있습니다.  
- **훈련된 에이전트를 Hub에 업로드하고**, 영상 리플레이와 평가 점수와 함께 공유할 수 있습니다 🔥.

## 이 노트북은 딥 강화학습 강의의 일부입니다
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

이 무료 강의에서는 다음을 배울 수 있습니다:

- 📖 **이론과 실습**을 통해 딥 강화학습(Deep Reinforcement Learning)을 학습합니다.  
- 🧑‍💻 Stable Baselines3, RL Baselines3 Zoo, CleanRL, Sample Factory 2.0 같은 **유명한 Deep RL 라이브러리 사용법**을 익힙니다.  
- 🤖 **다양한 환경에서 에이전트를 학습**시켜 봅니다.  

그리고 더 많은 내용을 원한다면 📚 **전체 커리큘럼은 여기서 확인하세요** 👉 https://simoninithomas.github.io/deep-rl-course

또한, 매 유닛이 공개될 때마다 링크와 도전 과제, 업데이트 내용을 받아보려면  
**[수강 신청](http://eepurl.com/ic5ZUD)** 을 꼭 해주세요!  
(이메일을 통해 정보를 보내드리기 위함입니다.)

커뮤니티 및 강사진과의 교류를 원한다면  
**디스코드 서버에 참여하세요** 👉🏻 https://discord.gg/ydHrjt3WP5

## 사전 준비 사항 🏗️  
노트북을 시작하기 전에 다음을 완료해야 합니다:

🔲 📚 [Unit 4를 읽고 Policy Gradient에 대해 학습하기](https://huggingface.co/deep-rl-course/unit4/introduction)

# Reinforce 알고리즘을 처음부터 직접 구현해보자 🔥

이 실습을 인증 과정에 제출하려면, 훈련한 모델을 Hub에 업로드해야 합니다.

- `Cartpole-v1`에서 **결과가 350 이상**일 것  
- `PixelCopter`에서 **결과가 5 이상**일 것

결과를 확인하려면 리더보드에 들어가서 자신의 모델을 찾으세요.  
**결과 = 평균 보상 - 보상 표준편차(mean_reward - std of reward)** 입니다.  
**리더보드에 자신의 모델이 보이지 않으면, 페이지 하단으로 가서 새로고침 버튼을 누르세요.**

인증 과정에 대한 자세한 정보는 다음 링크를 참고하세요 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## 팁 💡  
Colab에서 이 노트북을 실행할 때는 **Google Drive에 복사본을 만들어서 실행하는 것이 좋습니다.**  
그래야 **세션이 만료되더라도** 노트북이 Google Drive에 저장되어 다시 처음부터 작성할 필요가 없습니다.

복사 방법:  
`Ctrl + S` 또는 `파일 > Google 드라이브에 사본 저장` 을 클릭하세요.

## GPU 설정하기 💪  
- **에이전트의 훈련을 가속하기 위해 GPU를 사용할 예정입니다**. 이를 위해 `런타임 > 런타임 유형 변경`으로 이동하세요.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `하드웨어 가속기 > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 가상 디스플레이 생성 🔽

노트북을 실행하는 동안 리플레이 비디오를 생성해야 합니다. 이를 위해 코랩에서는 **환경을 렌더링(그리고 프레임을 기록)할 수 있도록 가상 화면이 필요합니다.**

따라서 다음 셀은 라이브러리를 설치하고 가상 화면 🖥을 생성 및 실행합니다.

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [ ]:
# 가상 디스플레이
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## 의존성 설치 🔽

첫 번째 단계는 의존성들을 설치하는 것입니다. 다음 패키지들을 설치합니다:

- `gym`  
- `gym-games`: PyGame으로 만들어진 추가 Gym 환경들  
- `huggingface_hub`: 🤗 모델과 데이터셋을 공유하고 탐색할 수 있는 중앙 허브. 버전 관리, 메트릭, 시각화 등 협업에 유용한 기능들을 제공합니다.

“왜 최신 버전인 gymnasium이 아니라 gym을 설치하나요?”라고 궁금할 수 있습니다.  
**우리가 사용하는 gym-games가 아직 gymnasium에 맞게 업데이트되지 않았기 때문입니다.**

두 라이브러리의 주요 차이점:
- `gym`에서는 `terminated`와 `truncated`가 없고, 대신 `done`만 존재합니다.  
- `env.step()`의 반환값이 `state, reward, done, info` 형식입니다.

Gym과 Gymnasium의 차이에 대한 자세한 내용은 여기를 참고하세요 👉 https://gymnasium.farama.org/content/migration-guide/

Reinforce 모델 모음 👉 https://huggingface.co/models?other=reinforce  
딥 강화학습 모델 전체 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

## 패키지 임포트 📦

설치한 라이브러리 외에도, 다음과 같은 패키지를 함께 임포트합니다:

- `imageio`: 리플레이 영상을 생성할 때 사용하는 라이브러리

In [ ]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # Hugging Face 계정에 로그인하여 모델을 Hub에 업로드할 수 있도록 합니다.
import imageio

## GPU가 있는지 확인하기

- GPU가 있는지 확인해봅시다.
- 만약 있다면 `device:cuda0`를 볼 수 있습니다.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

이제 Reinforce 알고리즘을 구현할 준비가 되었습니다 🔥

# 첫 번째 에이전트: CartPole-v1 게임 플레이 🤖

## CartPole 환경 생성 및 작동 방식 이해하기  
### [환경 🎮](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)

### 왜 CartPole-v1처럼 단순한 환경을 사용하는가?
[Reinforcement Learning Tips and Tricks](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)에서 설명한 것처럼, 에이전트를 처음부터 직접 구현할 때는 **깊이 있는 환경으로 들어가기 전에 간단한 환경에서 제대로 작동하는지 확인하고 버그를 찾아야 한다.** 간단한 환경에서는 버그를 찾는 것이 훨씬 쉽기 때문이다.

> 장난감 문제에서 "생존 신호"를 확인해보라

> 구현이 제대로 되었는지 확인하기 위해 점점 더 어려운 환경에서 실행해보라 (결과는 RL zoo와 비교 가능). 이 단계에서는 일반적으로 하이퍼파라미터 튜닝이 필요하다.

___
### CartPole-v1 환경

> 막대가 움직이지 않는 관절로 카트에 연결되어 있고, 카트는 마찰이 없는 트랙 위에서 움직인다. 진자(막대)는 카트 위에 수직으로 세워져 있으며, 목표는 카트를 왼쪽 또는 오른쪽으로 움직여 막대를 균형 상태로 유지하는 것이다.

그래서 우리는 CartPole-v1부터 시작한다. 목표는 **막대가 균형을 유지할 수 있도록 카트를 왼쪽 또는 오른쪽으로 밀어주는 것**이다.

에피소드는 다음 조건 중 하나라도 충족하면 종료된다:
- 막대의 각도가 ±12°를 초과할 때
- 카트의 위치가 ±2.4를 초과할 때
- 에피소드 길이가 500을 초과할 때

막대가 균형을 유지하는 매 시간마다 보상 💰 +1을 얻는다.

In [ ]:
env_id = "CartPole-v1"  
# 환경 생성  
env = gym.make(env_id)

# 평가용 환경 생성  
eval_env = gym.make(env_id)

# 상태 공간과 행동 공간 가져오기  
s_size = env.observation_space.shape[0]  
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("상태 공간의 크기:", s_size)
print("임의의 상태 샘플:", env.observation_space.sample())  # 랜덤 상태 가져오기

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("행동 공간의 크기:", a_size)
print("임의의 행동 샘플:", env.action_space.sample())  # 랜덤 행동 선택

## 이제 Reinforce 아키텍처를 구축해보자  
이 구현은 다음 두 가지 구현을 기반으로 한다:  
- [PyTorch 공식 강화 학습 예제](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)  
- [Udacity Reinforce](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)  
- [Chris1nexus에 의한 통합 개선](https://github.com/huggingface/deep-rl-class/pull/95)  

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce"/>

우리가 원하는 것은 다음과 같다:  
- 두 개의 완전 연결 계층 (fc1과 fc2)  
- fc1에는 ReLU 활성화 함수를 사용  
- fc2는 행동에 대한 확률 분포를 출력하기 위해 Softmax 사용

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 두 개의 완전 연결 레이어를 생성합니다.


    def forward(self, x):
        # 순방향 전달을 정의합니다.
        # 상태는 fc1으로 이동한 후 ReLU 활성화 함수를 적용합니다.

        # fc1 출력은 fc2로 이동합니다.

        # 소프트맥스를 출력합니다.

    def act(self, state):
        """
        주어진 상태에서 행동을 취합니다.
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

### 해결 방법

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

내가 실수했어, 어디서 했는지 맞춰볼 수 있겠어?

- 알아보려면 순방향 전파를 해보자:

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

- 여기서 오류 메시지가 `ValueError: The value argument to log_prob must be a Tensor` 라고 나타나는 것을 볼 수 있습니다.

- 이는 `m.log_prob(action)`에서 `action`이 Tensor여야 하지만, **그렇지 않다는 것을 의미합니다.**

- 이유를 아시겠습니까? act 함수를 확인하고 왜 작동하지 않는지 파악해 보세요.

조언 💡: 이 구현에는 무언가 잘못된 부분이 있습니다. act 함수에서 **행동의 확률 분포로부터 행동을 샘플링하기를 원한다는 것**을 기억하세요.

### (진짜) 해결 방법

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

CartPole을 사용함으로써 **버그가 간단한 환경이 아닌 우리의 통합 과정에서 발생했다는 것을 알기 때문에** 디버깅이 더 쉬웠습니다.

- **행동의 확률 분포로부터 행동을 샘플링하기를 원하기 때문에**, 항상 가장 높은 확률을 가진 행동만을 출력하는 `action = np.argmax(m)`를 사용할 수 없습니다.

- 확률 분포 P(.|s)에서 행동을 샘플링하는 `action = m.sample()`로 대체해야 합니다.

### Reinforce 훈련 알고리즘을 구축해 봅시다
다음은 Reinforce 알고리즘의 의사코드입니다:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_pseudocode.png" alt="Policy gradient pseudocode"/>
  

- 리턴 Gt (6행)를 계산할 때, **시점 t부터 시작하는** 할인된 보상의 합계를 계산한다는 것을 알 수 있습니다.

- 왜 그럴까요? 우리의 정책은 **결과에 근거하여 행동을 강화해야 하기 때문입니다**: 즉, 행동을 취하기 전에 얻은 보상은 소용이 없습니다 (그 행동 때문에 발생한 것이 아니므로), **행동 후에 오는 보상만이 중요합니다**.

- 이것을 코딩하기 전에 reward-to-go 정책 경사를 사용하는 이유를 설명하는 [과거에 현혹되지 마세요(don't let the past distract you)](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#don-t-let-the-past-distract-you) 섹션을 읽어보시는 것이 좋습니다.

우리는 각 시점의 리턴을 **효율적으로 계산하기 위해** [Chris1nexus](https://github.com/Chris1nexus)가 코딩한 흥미로운 기법을 사용합니다. 주석에 절차가 설명되어 있습니다. [PR 설명을 확인하는 것](https://github.com/huggingface/deep-rl-class/pull/95)도 주저하지 마세요.
하지만 전반적으로 아이디어는 **각 시점의 리턴을 효율적으로 계산하는 것**입니다.

두 번째로 궁금하실 점은 **왜 손실을 최소화하는가** 일 것입니다. 경사 하강(Gradient Descent)이 아닌 경사 상승(Gradient Ascent)에 대해 이야기했었는데요?

- 우리는 효용 함수 $J(\theta)$를 최대화하기를 원하지만, PyTorch는 TensorFlow처럼 **목적 함수를 최소화하는 것**이 더 좋습니다.
    - 특정 시점에서 행동 3을 강화한다고 가정해 봅시다. 훈련 전에 이 행동의 확률 P는 0.25입니다.
    - 따라서 우리는 $\theta$를 수정하여 $\pi_\theta(a_3|s; \theta) > 0.25$가 되도록 하고자 합니다.
    - 모든 P의 합은 1이 되어야 하므로, $\pi_\theta(a_3|s; \theta)$를 최대화하면 **다른 행동의 확률이 최소화됩니다.**
    - 따라서 PyTorch에게 **$1 - \pi_\theta(a_3|s; \theta)$를 최소화**하도록 지시해야 합니다.
    - 이 손실 함수는 $\pi_\theta(a_3|s; \theta)$가 1에 가까워짐에 따라 0에 수렴합니다.
    - 따라서 우리는 경사가 $\pi_\theta(a_3|s; \theta)$를 최대화하도록 유도하고 있습니다.

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # 훈련 중 점수 계산을 돕습니다.
    scores_deque = deque(maxlen=100)
    scores = []
    # 의사코드 3행
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state =  # TODO: 환경 리셋
        # 의사코드 4행
        for t in range(max_t):
            action, log_prob = # TODO 행동 가져오기
            saved_log_probs.append(log_prob)
            state, reward, done, _ = # TODO: 환경 단계 진행
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # 의사코드 6행: 리턴 계산
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        # 각 시점에서의 할인된 리턴을 계산합니다.
        # 시점 t에서의 감마-할인된 리턴 (G_t) + 시점 t에서의 보상 합계입니다.

        # O(N) 시간 복잡도로, 여기서 N은 시점의 수입니다.
        # (할인된 리턴 G_t의 이 정의는 Sutton&Barto 2017 2nd draft 44페이지에
        # 제시된 이 양의 정의를 따릅니다)
        # G_t = r_(t+1) + r_(t+2) + ...

        # 이 공식이 주어졌을 때, 각 시점 t에서의 리턴은
        # 계산된 미래 리턴 G_(t+1)을 재사용하여 현재 리턴 G_t를 계산함으로써
        # 계산될 수 있습니다.
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (이는 동적 계획법 접근 방식을 따르며, 여러 번 계산하는 것을 피하기 위해
        # 해를 기억합니다)

        # 위는 다음 식과 동일하므로 올바릅니다 (Sutton&Barto 2017 2nd draft 46페이지 참조)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...


        ## 위 내용을 바탕으로 시점 t에서의 리턴을 다음과 같이 계산합니다:
        #           gamma[t] * return[t] + reward[t]
        #
        ## 위에서 제시된 공식을 사용하고, 처음부터 마지막까지 계산할 때 필요한
        ## 중복 계산을 피하기 위해 마지막 시점부터 첫 시점까지 계산합니다.

        ## 따라서 "returns" 큐는 appendleft() 함수 덕분에 시간복잡도 O(1)로
        ## 위치 0에 추가할 수 있어 t=0부터 t=n_steps까지 시간 순서대로 리턴을 저장합니다.
        ## 일반적인 파이썬 리스트는 이를 위해 O(N)이 필요합니다.

        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(    ) # TODO: 여기 완성
        
        ## 훈련을 더 안정적으로 만들기 위해 리턴의 표준화가 사용됩니다.
        eps = np.finfo(np.float32).eps.item()

        ## eps는 가장 작은 표현 가능한 float이며,
        # 수치적 불안정성을 피하기 위해 리턴의 표준 편차에 더해집니다.
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        # 7행:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        # 8행: PyTorch는 경사 하강을 선호합니다.
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('에피소드 {}\t평균 점수: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores


#### 해결 방법

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # 훈련 중 점수 계산을 돕습니다.
    scores_deque = deque(maxlen=100)
    scores = []
    # 의사코드 3행
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset() # TODO: 환경 리셋
        # 의사코드 4행
        for t in range(max_t):
            action, log_prob = policy.act(state) # TODO 행동 가져오기
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action) # TODO: 환경 단계 진행
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # 의사코드 6행: 리턴 계산
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        # 각 시점에서의 할인된 리턴을 계산합니다.
        # 시점 t에서의 감마-할인된 리턴 (G_t) + 시점 t에서의 보상 합계입니다.

        # O(N) 시간 복잡도로, 여기서 N은 시점의 수입니다.
        # (할인된 리턴 G_t의 이 정의는 Sutton&Barto 2017 2nd draft 44페이지에
        # 제시된 이 양의 정의를 따릅니다)
        # G_t = r_(t+1) + r_(t+2) + ...

        # 이 공식이 주어졌을 때, 각 시점 t에서의 리턴은
        # 계산된 미래 리턴 G_(t+1)을 재사용하여 현재 리턴 G_t를 계산함으로써
        # 계산될 수 있습니다.
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (이는 동적 계획법 접근 방식을 따르며, 여러 번 계산하는 것을 피하기 위해
        # 해를 기억합니다)

        # 위는 다음 식과 동일하므로 올바릅니다 (Sutton&Barto 2017 2nd draft 46페이지 참조)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...


        ## 위 내용을 바탕으로 시점 t에서의 리턴을 다음과 같이 계산합니다:
        #           gamma[t] * return[t] + reward[t]
        #
        ## 위에서 제시된 공식을 사용하고, 처음부터 마지막까지 계산할 때 필요한
        ## 중복 계산을 피하기 위해 마지막 시점부터 첫 시점까지 계산합니다.

        ## 따라서 "returns" 큐는 appendleft() 함수 덕분에 시간복잡도 O(1)로
        ## 위치 0에 추가할 수 있어 t=0부터 t=n_steps까지 시간 순서대로 리턴을 저장합니다.
        ## 일반적인 파이썬 리스트는 이를 위해 O(N)이 필요합니다.

        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( disc_return_t * gamma + rewards[t] ) # TODO: 여기 완성
        
        ## 훈련을 더 안정적으로 만들기 위해 리턴의 표준화가 사용됩니다.
        eps = np.finfo(np.float32).eps.item()

        ## eps는 가장 작은 표현 가능한 float이며,
        # 수치적 불안정성을 피하기 위해 리턴의 표준 편차에 더해집니다.
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        # 7행:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        # 8행: PyTorch는 경사 하강을 선호합니다.
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('에피소드 {}\t평균 점수: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores


##  학습 시작  
- 이제 에이전트를 학습시킬 준비가 되었다.  
- 먼저, 모든 학습 하이퍼파라미터를 담은 변수를 정의한다.  
- 학습 파라미터는 변경할 수 있으며, 변경하는 것이 좋다 😉

In [ ]:
cartpole_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [ ]:
# 정책 생성 및 디바이스에 배치
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [ ]:
# TypeError: expected np.ndarray (got tuple) 발생 시 gym<0.26 버전 설치
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"], 
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"], 
                   100)

## 평가 방법 정의 📝
- 여기서 Reinforce 에이전트를 테스트하는 데 사용할 평가 방법을 정의합니다.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  에이전트를 ``n_eval_episodes`` 에피소드 동안 평가하고 평균 보상과 보상의 표준 편차를 반환합니다.
  :param env: 평가 환경
  :param n_eval_episodes: 에이전트를 평가할 에피소드 수
  :param policy: Reinforce 에이전트
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward
        
      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## 에이전트 평가하기 📈

In [ ]:
evaluate_agent(eval_env, 
               cartpole_hyperparameters["max_t"], 
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

### 학습된 모델을 Hub에 게시하기 🔥
이제 학습 후에 좋은 결과를 얻었으므로, 한 줄의 코드로 학습된 모델을 hub 🤗에 게시할 수 있습니다.

다음은 모델 카드 예시입니다:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/modelcard.png"/>

### 허브에 푸시  
#### 이 코드는 수정하지 마세요

In [ ]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

In [ ]:
def record_video(env, policy, out_directory, fps=30):
  """
  에이전트의 리플레이 영상을 생성합니다
  :param env: 환경
  :param policy: 에이전트의 정책
  :param out_directory: 출력 디렉토리
  :param fps: 초당 프레임 수 (taxi-v3와 frozenlake-v1에서는 1 사용)
  """
  images = []  
  done = False
  state = env.reset()
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # 해당 상태에서 기대되는 미래 보상이 가장 큰 행동(인덱스)을 선택
    action, _ = policy.act(state)
    state, reward, done, info = env.step(action)  # 녹화 로직을 위해 다음 상태를 state로 바로 저장
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
def push_to_hub(repo_id, 
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):
  """
  모델을 평가하고, 영상 생성 후 Hugging Face Hub에 업로드합니다.
  이 메서드는 전체 파이프라인을 수행합니다:
  - 모델을 평가합니다
  - 모델 카드를 생성합니다
  - 에이전트의 리플레이 영상을 생성합니다
  - 모든 결과를 Hub에 업로드합니다

  :param repo_id: Hugging Face Hub의 모델 저장소 ID
  :param model: 저장할 PyTorch 모델
  :param hyperparameters: 학습 하이퍼파라미터
  :param eval_env: 평가 환경
  :param video_fps: 리플레이 영상 녹화 시 초당 프레임 수
  """

  _, repo_name = repo_id.split("/")
  api = HfApi()
  
  # 1단계: 저장소 생성
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)
  
    # 2단계: 모델 저장
    torch.save(model, local_directory / "model.pt")

    # 3단계: 하이퍼파라미터를 JSON 파일로 저장
    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(hyperparameters, outfile)
    
    # 4단계: 모델 평가 및 결과 저장
    mean_reward, std_reward = evaluate_agent(eval_env, 
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"], 
                                            model)
    # 평가 시각 저장
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"], 
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    # JSON 파일로 저장
    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # 5단계: 모델 카드 생성
    env_name = hyperparameters["env_id"]
    
    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    # 평가 지표 추가
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    # 메타데이터 병합
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** Agent playing **{env_id}**
  This is a trained model of a **Reinforce** agent playing **{env_id}** .
  To learn to use this model and train yours check Unit 4 of the Deep Reinforcement Learning Course: https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    # 메타데이터를 README에 저장
    metadata_save(readme_path, metadata)

    # 6단계: 영상 녹화
    video_path =  local_directory / "replay.mp4"
    record_video(env, model, video_path, video_fps)

    # 7단계: Hub에 업로드
    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"모델이 Hub에 업로드되었습니다. 여기에서 확인할 수 있습니다: {repo_url}")

### .

`push_to_hub`을 사용하면 **에이전트를 평가하고, 리플레이를 녹화하며, 모델 카드를 생성한 뒤 Hub에 업로드할 수 있습니다**.

이렇게 하면:
- **작업 결과를 멋지게 보여줄 수 있습니다** 🔥  
- **에이전트가 플레이하는 모습을 시각화할 수 있습니다** 👀  
- **다른 사람들이 사용할 수 있도록 에이전트를 커뮤니티에 공유할 수 있습니다** 💾  
- **클래스메이트들과 비교해 에이전트의 성능을 리더보드에서 확인할 수 있습니다** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

커뮤니티와 모델을 공유하려면 다음의 세 가지 단계를 추가로 따라야 합니다:

---

1️⃣ **(아직 하지 않았다면)** Hugging Face 계정을 생성하세요 ➡ https://huggingface.co/join

2️⃣ 로그인한 후, Hugging Face 웹사이트에서 인증 토큰을 저장해야 합니다.  
- 새 토큰을 생성하세요 (https://huggingface.co/settings/tokens)  
  **권한은 *쓰기(write)* 로 설정해야 합니다**

---

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

In [ ]:
notebook_login()

Google Colab이나 Jupyter Notebook을 사용하지 않는 경우, 아래 명령어를 대신 사용해야 합니다: `huggingface-cli login`

3️⃣ 이제 훈련된 에이전트를 🤗 허브에 푸시할 준비가 되었습니다 🔥

In [ ]:
repo_id = ""  # TODO: 자신의 저장소 ID를 정의하세요. 형식: {username}/Reinforce-{model_id}
push_to_hub(
    repo_id,
    cartpole_policy,            # 저장할 모델
    cartpole_hyperparameters,   # 하이퍼파라미터
    eval_env,                   # 평가 환경
    video_fps=30                # 리플레이 영상의 FPS
)

이제 구현의 견고함을 테스트해봤으니, 좀 더 복잡한 환경에 도전해봅시다: **PixelCopter** 🚁

## 두 번째 에이전트: PixelCopter 🚁

### PixelCopter 환경 살펴보기 👀  
- [환경 문서 보기](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)  

In [ ]:
env_id = "Pixelcopter-PLE-v0"
env = gym.make(env_id)
eval_env = gym.make(env_id)
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("상태 공간의 크기:", s_size)
print("임의의 상태 샘플:", env.observation_space.sample())  # 랜덤 상태 가져오기

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("행동 공간의 크기:", a_size)
print("임의의 행동 샘플:", env.action_space.sample())  # 랜덤 행동 선택

관찰 공간 (7) 👀:  
- 플레이어의 y 위치  
- 플레이어의 속도  
- 플레이어와 바닥 간 거리  
- 플레이어와 천장 간 거리  
- 다음 블록까지의 x 거리  
- 다음 블록의 상단 y 위치  
- 다음 블록의 하단 y 위치  

행동 공간 (2) 🎮:  
- 위로 (가속기 누르기)  
- 아무것도 하지 않기 (가속기 누르지 않기)  

보상 함수 💰:  
- 수직 블록 하나를 통과할 때마다 +1의 양의 보상을 얻음.  
- 터미널 상태에 도달할 때마다 -1의 음의 보상을 받음.

### 새로운 정책 정의 🧠  
- 환경이 더 복잡하므로 더 깊은 신경망이 필요함

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 세 개의 레이어 정의
        
    def forward(self, x):
        # 순전파 과정 정의 
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

#### 해결 방법

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size)
        self.fc3 = nn.Linear(h_size, a_size)
        self.swish = nn.SiLU()

    def forward(self, x):
        x = self.swish(self.fc1(x))
        x = self.swish(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

### 하이퍼파라미터 정의 ⚙️  
- 이 환경은 더 복잡하므로 하이퍼파라미터를 조정해야 함  
- 특히 은닉층 크기는 더 많은 뉴런이 필요함  

In [ ]:
pixelcopter_hyperparameters = {
    "h_size": 128,
    "n_training_episodes": 50000,
    "n_evaluation_episodes": 10,
    "max_t": 10000,
    "gamma": 0.99,
    "lr": 5e-5,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

### 훈련 시작  
- 이제 에이전트를 훈련할 준비가 되었다 🔥.

In [ ]:
# 정책 생성 및 디바이스에 배치
torch.manual_seed(50)
pixelcopter_policy = Policy(pixelcopter_hyperparameters["state_space"], pixelcopter_hyperparameters["action_space"], pixelcopter_hyperparameters["h_size"]).to(device)
pixelcopter_optimizer = optim.Adam(pixelcopter_policy.parameters(), lr=pixelcopter_hyperparameters["lr"])

In [ ]:
scores = reinforce(pixelcopter_policy,
                   pixelcopter_optimizer,
                   pixelcopter_hyperparameters["n_training_episodes"], 
                   pixelcopter_hyperparameters["max_t"],
                   pixelcopter_hyperparameters["gamma"], 
                   1000)

In [ ]:
mean_reward, std_reward = evaluate_agent(eval_env, 
                                            pixelcopter_hyperparameters["max_t"],
                                            pixelcopter_hyperparameters["n_evaluation_episodes"], 
                                            pixelcopter_policy)

In [ ]:
print(mean_reward)

### 학습된 모델을 허브에 게시하세요 🔥

In [ ]:
repo_id = "" #TODO 자신의 저장소 ID를 정의하세요 {username/Reinforce-{model-id}}
push_to_hub(
    repo_id,
    pixelcopter_policy,          # 저장할 모델
    pixelcopter_hyperparameters, # 하이퍼파라미터
    eval_env,                    # 평가 환경
    video_fps=30
)

## 몇 가지 추가 도전 과제 🏆
가장 좋은 학습 방법은 **직접 시도해보는 것**입니다! 보셨듯이 현재 에이전트의 성능이 좋지 않습니다. 첫 번째 제안으로는 더 많은 단계를 학습할 수 있습니다. 또한 더 나은 파라미터를 찾아보세요.

[리더보드](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard)에서 여러분의 에이전트를 찾을 수 있습니다. 정상에 오를 수 있을까요?

이를 달성하기 위한 몇 가지 아이디어는 다음과 같습니다.
* 더 많은 단계 학습
* 동료들이 무엇을 했는지 보면서 다른 하이퍼파라미터를 시도해 보세요 👉 https://huggingface.co/models?other=reinforce
* **새롭게 학습된 모델을** Hub에 푸시하기 🔥
* **더 복잡한 환경에 대한 구현 개선** (예를 들어, 관찰값으로 프레임을 처리하기 위해 네트워크를 컨볼루션 신경망으로 변경하는 것은 어떨까요?)

________________________________________________________________________

**이 유닛을 마치신 것을 축하드립니다**! 많은 정보가 있었습니다.
그리고 튜토리얼을 마치신 것도 축하드립니다. PyTorch를 사용하여 첫 번째 심층 강화 학습 에이전트를 처음부터 코딩하고 Hub에 공유하셨습니다 🥳.

**더 복잡한 환경에 대한 구현을 개선하여** 이 유닛을 반복하는 것을 주저하지 마세요 (예를 들어, 관찰값으로 프레임을 처리하기 위해 네트워크를 컨볼루션 신경망으로 변경하는 것은 어떨까요?).

다음 유닛에서는 Unity 환경에서 에이전트를 학습시키면서 **Unity MLAgents에 대해 더 자세히 배울 것입니다**. 이로써 **눈싸움과 축구 게임에서 다른 에이전트들과 경쟁하기 위해 에이전트를 학습시키는 AI 대 AI 챌린지**에 참여할 준비가 될 것입니다.

재미있을 것 같나요? 다음에 만나요!

마지막으로, **강좌에 대해 어떻게 생각하시는지, 어떻게 개선할 수 있는지** 듣고 싶습니다. 피드백이 있으시면 👉 [이 양식을 작성해주세요](https://forms.gle/BzKXWzLAGZESGNaE9)

유닛 5에서 만나요! 🔥

### 계속 배우고, 멋지게 지내세요 🤗